In [2]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

# 4 nodes

In [3]:
df_all = []
for i in range(4):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/calculating_scores/cfl_4_noiid/fedstellar_CFL_16_07_2024_19_22_58/metrics/participant_{i}"
    date_str = "02/07/2024 10:22:20" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step      value                     timestamp                    tag
0   635  38.900002 2024-07-16 17:27:14.651591168  Resources/CPU_percent
1   645  39.700001 2024-07-16 17:27:24.711831296  Resources/CPU_percent
2   655  37.599998 2024-07-16 17:27:34.734873344  Resources/CPU_percent
3   666  39.500000 2024-07-16 17:27:45.457054720  Resources/CPU_percent
4   676  37.799999 2024-07-16 17:27:55.507958016  Resources/CPU_percent
2024-07-16 17:38:40 2024-07-16 17:23:14
                step     value                     timestamp
tag                                                         
Test/Accuracy   5359  0.109182 2024-07-16 17:38:39.129810944
Test/F1Score    5359  0.021760 2024-07-16 17:38:39.129840128
Test/Loss       5359       NaN 2024-07-16 17:38:39.129774336
Test/Precision  5359  0.012371 2024-07-16 17:38:39.129821696
Test/Recall     5359  0.109182 2024-07-16 17:38:39.129831168
   step  value                     timestamp         tag
0   746    NaN 2024-07-16 17:28:28.438684672

In [4]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5359,0.109182,2024-07-16 17:38:39.129810944,participant_0
Test/F1Score,5359,0.021760,2024-07-16 17:38:39.129840128,participant_0
Test/Loss,5359,NaN,2024-07-16 17:38:39.129774336,participant_0
Test/Precision,5359,0.012371,2024-07-16 17:38:39.129821696,participant_0
Test/Recall,5359,0.109182,2024-07-16 17:38:39.129831168,participant_0
Test/Accuracy,5402,0.110840,2024-07-16 17:38:39.147833600,participant_1
Test/F1Score,5402,0.021316,2024-07-16 17:38:39.147862272,participant_1
Test/Loss,5402,NaN,2024-07-16 17:38:39.147806208,participant_1
Test/Precision,5402,0.012051,2024-07-16 17:38:39.147844352,participant_1


In [5]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_73170/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,5420.75,0.1104
Test/F1Score,5420.75,0.0219
Test/Loss,5420.75,120.2416
Test/Precision,5420.75,0.0124
Test/Recall,5420.75,0.1104


In [8]:
concat_df.to_csv("results_csvs/dfl_15_star.csv")